In [1]:
import pandas as pd
import os
import datetime

In [2]:
noticias_csv_path = 'data/processed/pruebaV1.csv' # Ajusta esta ruta
df = pd.read_csv(noticias_csv_path)

In [3]:
import pandas as pd

# Asegúrate de que la columna sea datetime
df["publish_date"] = pd.to_datetime(df["publish_date"], errors="coerce")

# Seleccionar las 10.000 filas más antiguas
df_antiguas = df.nsmallest(10000, "publish_date")



In [4]:
df_antiguas

,Unnamed: 0,id,title,source,country,publish_date,platform,title_normalized,personas_title,personas_title_count,body,year_month,n_personas
0,182940,1073f805cdc17e54d7ff9de18301a2dee8afa5fb32930f...,José Maza intentará batir un récord Guinness c...,elmostrador.cl,Chile,2019-07-01,Event Registry,josé maza intentará batir un récord guinness c...,['José Maza'],1,"Se estableció en 2018 en Vermont, Estados Unid...",2019-07,1
1,182551,40abf2b284f0f13adfa27720e930893cc6da109750a498...,PiÃ±era tilda paro de profesores como ilegal y...,biobiochile.cl,Chile,2019-07-01,Event Registry,piã±era tilda paro de profesores como ilegal y...,['PiÃ±era'],1,Previo a su retorno al paÃ­s tras la gira que ...,2019-07,1
2,183041,b3ae8649ce749529f4dc7790f38890962e5df7d02ac644...,"Jorge Aravena: ""Siempre le preguntaba a Charle...",lacuarta.com,Chile,2019-07-01,Event Registry,"jorge aravena: ""siempre le preguntaba a charle...","['Charles', 'Jorge Aravena']",2,"Aránguiz recordó al ""Mortero"" por una frase qu...",2019-07,2
3,182956,63186f238015f6ec5ece5502098f4485400e8f31daff4a...,Ganó el Sí: se impone el toque de queda infant...,elmostrador.cl,Chile,2019-07-01,Event Registry,ganó el sí: se impone el toque de queda infant...,"['Ganó el Sí', 'Lavín']",2,Pasado las 18:00 horas de este domingo concluy...,2019-07,2
4,182902,9bbf29706f13163df8e16442f8f65fe7fa85c811117870...,Sigue el drama socialista: Landerretche sugier...,elmostrador.cl,Chile,2019-07-01,Event Registry,sigue el drama socialista: landerretche sugier...,"['Landerretche', 'Maya Fernández', 'Álvaro Eli...",3,El Partido Socialista entró en hora de definic...,2019-07,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,676203,31a38921bd270b1538962063783d187d6e3f421459ecba...,La irresponsabilidad de Zaldivia y la habilida...,latercera.com,Chile,2021-04-12,Event Registry,la irresponsabilidad de zaldivia y la habilida...,"['Castro', ""O'Higgins""]",2,La expulsión del defensor fue demasiado para l...,2021-04,2
9996,675225,3681bce79f4204aa09d999d28a828e4dcd02efbeaa51d2...,Presidente Piñera y otros líderes latinoameric...,24horas.cl,Chile,2021-04-12,Event Registry,presidente piñera y otros líderes latinoameric...,"['Guillermo Lasso', 'Piñera']",2,El candidato por la alianza conservadora CREO-...,2021-04,2
9997,675364,cf109a7efff3b532eae140e33cdbb7ba1732f4627b6c58...,"Pensemos Chile: Jaime Mañalich, Axel Callís y ...",adnradio.cl,Chile,2021-04-12,Event Registry,"pensemos chile: jaime mañalich, axel callís y ...","['Axel Callís', 'Jaime Mañalich', 'Luz Poblete']",3,"{""multiple"":false,""videos"":[{""key"":""CQgJSXzyTp...",2021-04,3
9998,675752,750afc82a7d50ca9c0572bd3438a8e80797471274e48cf...,Vicente Painel Seguel acusa al Poder judicial ...,elciudadano.com,Chile,2021-04-12,Event Registry,vicente painel seguel acusa al poder judicial ...,"['Fuad Chahín', 'Vicente Painel Seguel']",2,Continúan las repercusiones por la querella pr...,2021-04,2


In [5]:
# Celda 2: Preparación de Prompt, Nodos Semilla y Prueba de Formato

import json 
import os   


df_filtrado = df_antiguas.copy()

# --- Función Auxiliar para Cargar Prompt ---
def load_prompt_from_file(filename: str) -> str:
    """Carga el prompt desde una ruta específica."""
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"Error CRÍTICO: El archivo de prompt '{filename}' no se encontró.")
        raise
    except Exception as e:
        print(f"Error CRÍTICO al leer el archivo de prompt '{filename}': {e}")
        raise

# --- Preparación del Prompt y Nodos Semilla ---
prompt_file_path = 'prompts/promptV1.txt'
base_prompt_content = load_prompt_from_file(prompt_file_path)
final_system_prompt = base_prompt_content


In [6]:
import json
import pandas as pd

# --- Generación de Tareas para el Batch ---
print("\nGenerando lista de tareas para el batch...")
batch_tasks_list = []
for index_original, row_data in df_filtrado.iterrows():
    noticia_description = str(row_data['body']) if pd.notna(row_data['body']) else ""
    if not noticia_description.strip():
        continue

    task_item = {
        "custom_id": f"task-{index_original}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-5-mini",
            "response_format": {"type": "json_object"},
            "messages": [
                {"role": "system", "content": final_system_prompt},
                {"role": "user", "content": noticia_description}
            ],
        }
    }
    batch_tasks_list.append(task_item)

print(f"Número total de tareas generadas para el batch: {len(batch_tasks_list)}")



Generando lista de tareas para el batch...
Número total de tareas generadas para el batch: 10000


In [10]:
# --- Celda B: Mostrar solo resumen de la primera tarea ---
if batch_tasks_list:
    primera_tarea = batch_tasks_list[0]

    print("📌 Primera tarea generada:")
    print(f"custom_id   : {primera_tarea['custom_id']}")
    print(f"method      : {primera_tarea['method']}")
    print(f"url         : {primera_tarea['url']}")
    print(f"modelo      : {primera_tarea['body']['model']}")
    print(f"sistema     : {primera_tarea['body']['messages'][0]['content']}...")  # primer 100 chars
    print(f"usuario     : {primera_tarea['body']['messages'][1]['content']}...")  # primer 200 chars
else:
    print("No hay tareas en batch_tasks_list")


📌 Primera tarea generada:
custom_id   : task-0
method      : POST
url         : /v1/chat/completions
modelo      : gpt-5-mini
sistema     : 
**Role:** Expert political analyst & network scientist.

**Mission:** From the provided Chilean political news, extract the main power network as a single JSON object.

**Key Constraints:**

1.  **Focus on Importance:** Identify and extract only the **most significant power relationships (edges)** from the news. 
2.  **Full Names Only:** Node labels MUST be the person's full name. **Strictly exclude any political or professional titles** (e.g., Presidente, Ministro, Diputada, Senador, Fiscal).
3.  **Individuals Only:** **Nodes must represent individual people. Strictly exclude all institutions, organizations, political parties, or collective bodies (e.g., "Cámara de Diputados", "Partido Republicano", "Gobierno", "Frente Amplio"). If an interaction is between an individual and an institution, or between two institutions, it must be ignored.**
**Out

In [11]:
# Celda 3: Generación de Tareas y Envío del Batch a OpenAI

import json
import io
import datetime
from openai import OpenAI # Re-importar o asegurarse de que el cliente esté disponible
from dotenv import load_dotenv

load_dotenv()


try:
    client = OpenAI()
    print("Cliente OpenAI inicializado/confirmado.")
except Exception as e:
    print(f"Error CRÍTICO al inicializar el cliente de OpenAI: {e}")
    raise

# --- Generación de Tareas para el Batch ---
print("\nGenerando lista de tareas para el batch...")
batch_tasks_list = []
for index_original, row_data in df_filtrado.iterrows(): # `index_original` es el índice de df_filtrado
    noticia_description = str(row_data['body']) if pd.notna(row_data['body']) else ""
    

    if not noticia_description.strip(): 
        continue

    task_item = {
        "custom_id": f"task-{index_original}", # Usa el índice original de la noticia
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-5-mini", # Asegúrate que este es el modelo deseado
            "response_format": {"type": "json_object"},
            "messages": [
                {"role": "system", "content": final_system_prompt},
                {"role": "user", "content": noticia_description}
            ],
        }
    }
    batch_tasks_list.append(task_item)

print(f"Número total de tareas generadas para el batch: {len(batch_tasks_list)}")

# --- Creación del Archivo Batch en Memoria y Envío ---
if not batch_tasks_list:
    print("No se generaron tareas. El batch no será enviado.")
else:
    # Confirmación antes de enviar (opcional, pero recomendado)
    user_confirmation_envio = input(f"Se han generado {len(batch_tasks_list)} tareas. ¿Deseas proceder con el envío del batch a OpenAI? (s/N): ")
    if user_confirmation_envio.lower() != 's':
        print("Envío del batch cancelado por el usuario.")
    else:
        print("\nProcediendo con la creación del archivo batch y envío a OpenAI...")
        jsonl_content_list_for_batch = [json.dumps(obj) for obj in batch_tasks_list]
        jsonl_content_str_for_batch = '\n'.join(jsonl_content_list_for_batch)

        batch_file_in_memory = io.BytesIO(jsonl_content_str_for_batch.encode('utf-8'))
        batch_file_in_memory.name = f"batch_input_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.jsonl"

        print(f"Creando archivo batch '{batch_file_in_memory.name}' en OpenAI...")
        try:
            openai_batch_file_object = client.files.create(
                file=batch_file_in_memory,
                purpose="batch"
            )
            print(f"Archivo batch creado en OpenAI con ID: {openai_batch_file_object.id}")

            print("Enviando job batch a OpenAI...")
            openai_batch_job_object = client.batches.create(
                input_file_id=openai_batch_file_object.id,
                endpoint="/v1/chat/completions",
                completion_window="24h" # Tiempo para que OpenAI complete el batch
            )
            print(f"Job batch enviado con ID: {openai_batch_job_object.id}")
            print("Puedes monitorear el estado del batch en el dashboard de OpenAI o usando la API.")
            print(f"Una vez completado, descarga el archivo de resultados JSONL (output).")
            print(f"Asegúrate de tener 'df_filtrado' (con {len(df_filtrado)} filas) disponible en memoria si Celda 4 lo necesita directamente.")

        except Exception as e:
            print(f"Error CRÍTICO durante la creación o envío del batch a OpenAI: {e}")
            # Considera `raise e` si el error debe detener todo flujo posterior

Cliente OpenAI inicializado/confirmado.

Generando lista de tareas para el batch...
Número total de tareas generadas para el batch: 10000

Procediendo con la creación del archivo batch y envío a OpenAI...
Creando archivo batch 'batch_input_20250925_003822.jsonl' en OpenAI...
Archivo batch creado en OpenAI con ID: file-QrMwUtKRXMXNiziXuFFnUv
Enviando job batch a OpenAI...
Job batch enviado con ID: batch_68d4b956f7ec8190a2eff7dc12efcbc0
Puedes monitorear el estado del batch en el dashboard de OpenAI o usando la API.
Una vez completado, descarga el archivo de resultados JSONL (output).
Asegúrate de tener 'df_filtrado' (con 10000 filas) disponible en memoria si Celda 4 lo necesita directamente.


# Parte 2

In [7]:
df

,Unnamed: 0,id,title,source,country,publish_date,platform,title_normalized,personas_title,personas_title_count,body,year_month,n_personas
0,182940,1073f805cdc17e54d7ff9de18301a2dee8afa5fb32930f...,José Maza intentará batir un récord Guinness c...,elmostrador.cl,Chile,2019-07-01,Event Registry,josé maza intentará batir un récord guinness c...,['José Maza'],1,"Se estableció en 2018 en Vermont, Estados Unid...",2019-07,1
1,182551,40abf2b284f0f13adfa27720e930893cc6da109750a498...,PiÃ±era tilda paro de profesores como ilegal y...,biobiochile.cl,Chile,2019-07-01,Event Registry,piã±era tilda paro de profesores como ilegal y...,['PiÃ±era'],1,Previo a su retorno al paÃ­s tras la gira que ...,2019-07,1
2,183041,b3ae8649ce749529f4dc7790f38890962e5df7d02ac644...,"Jorge Aravena: ""Siempre le preguntaba a Charle...",lacuarta.com,Chile,2019-07-01,Event Registry,"jorge aravena: ""siempre le preguntaba a charle...","['Charles', 'Jorge Aravena']",2,"Aránguiz recordó al ""Mortero"" por una frase qu...",2019-07,2
3,182956,63186f238015f6ec5ece5502098f4485400e8f31daff4a...,Ganó el Sí: se impone el toque de queda infant...,elmostrador.cl,Chile,2019-07-01,Event Registry,ganó el sí: se impone el toque de queda infant...,"['Ganó el Sí', 'Lavín']",2,Pasado las 18:00 horas de este domingo concluy...,2019-07,2
4,182902,9bbf29706f13163df8e16442f8f65fe7fa85c811117870...,Sigue el drama socialista: Landerretche sugier...,elmostrador.cl,Chile,2019-07-01,Event Registry,sigue el drama socialista: landerretche sugier...,"['Landerretche', 'Maya Fernández', 'Álvaro Eli...",3,El Partido Socialista entró en hora de definic...,2019-07,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36649,2863478,cdf3b162af236bb96e35cf8d38470db8a4d8b8c0e2a3f5...,"Trío ultra hot, ravotril y mea culpa: el acalo...",lacuarta.com,Chile,2025-09-21,Event Registry,"trío ultra hot, ravotril y mea culpa: el acalo...","['Kel Calderón', 'Vale Roth']",2,"El año 2012, se registró uno de los momentos d...",2025-09,2
36650,2863464,d74d5c414bfd7bec1f78aef3c906f4ca95b50b056fcd61...,Pedro Campos respaldó dichos de Claudia di Gir...,lacuarta.com,Chile,2025-09-21,Event Registry,pedro campos respaldó dichos de claudia di gir...,"['Claudia di Girolamo', 'Cristián Campos', 'Pe...",3,Este mes la Corte Suprema ratificó el fallo de...,2025-09,3
36651,2863177,6b29a472b1b01d1fffbcca4e080b5d1065439590b7614d...,"Ximena Rincón y rechazo de su candidatura: ""Se...",biobiochile.cl,Chile,2025-09-21,Event Registry,"ximena rincón y rechazo de su candidatura: ""se...","['Ximena Rincón', 'Álvaro Elizalde']",2,La senadora Ximena Rincón criticó nuevamente l...,2025-09,2
36652,2863525,49e8fbe1d559799ecee1db1648604c961d6470c3a20718...,Multitudinarias protestas en Brasil contra pos...,latercera.com,Chile,2025-09-21,Event Registry,multitudinarias protestas en brasil contra pos...,"['Bolsonaro', 'Multitudinarias']",2,Miles de personas realizaron protestas este do...,2025-09,2


In [10]:
# Celda 2: Procesamiento de Resultados y Generacion de Edgelist (Formato Modificado)

# --- Importaciones para Celda 2 ---
import pandas as pd
import json
from typing import List, Dict, Union, Optional

# --- Configuracion ---
# !!! IMPORTANTE: ACTUALIZA ESTA RUTA al archivo de resultados .jsonl descargado de OpenAI !!!
jsonl_file_path = "json/batch_68d4b956f7ec8190a2eff7dc12efcbc0_output.jsonl"  # Actualiza esta ruta

# Se asume que 'df' proviene de la celda anterior (df original con las noticias)
df_filtrado = df.copy()

# Nombre del archivo CSV de salida para el edgelist final
output_edgelist_csv_path = "output/flatline.csv"

# Nombres de las columnas en el DataFrame original
NOMBRE_COLUMNA_FECHA_NOTICIA = "publish_date"
NOMBRE_COLUMNA_TITULO_NOTICIA = "title"
NOMBRE_COLUMNA_CUERPO_NOTICIA = "body"
NOMBRE_COLUMNA_ID_NOTICIA = "id"

# --- Funciones Auxiliares ---
def read_jsonl_results(file_path: str) -> List[Dict]:
    '''Lee un archivo JSONL con los resultados del batch y devuelve una lista de dicts.'''
    json_objects: List[Dict] = []
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            for line_number, line in enumerate(file, 1):
                line_stripped = line.strip()
                if not line_stripped:
                    continue
                try:
                    json_objects.append(json.loads(line_stripped))
                except json.JSONDecodeError as exc:
                    print(
                        f"Advertencia: Error decodificando JSON en linea {line_number} "
                        f"del archivo '{file_path}': {exc}. Fragmento: '{line_stripped[:100]}...'"
                    )
    except FileNotFoundError:
        print(f"Error critico: El archivo '{file_path}' no se encontro. Verifica la ruta.")
        raise
    except Exception as exc:
        print(f"Error critico al leer el archivo '{file_path}': {exc}")
        raise
    return json_objects


def extract_network_edgelist_from_results_v2(
    df_with_results: pd.DataFrame,
    json_result_column: str,
    date_column_name: str,
    title_column_name: str,
    body_column_name: str,
    id_column_name: str,
) -> pd.DataFrame:
    '''Extrae un edgelist extendido desde una columna con resultados JSON.'''

    def parse_individual_json_result(data: Union[str, Dict, List, None]) -> Optional[Dict]:
        if data is None:
            return None
        if isinstance(data, dict):
            return data
        if isinstance(data, list):
            text_parts: List[str] = []
            for entry in data:
                if isinstance(entry, dict):
                    if "text" in entry and entry["text"]:
                        text_parts.append(entry["text"])
                    elif entry.get("type") == "output_text" and entry.get("content"):
                        text_parts.append(entry["content"])
            if text_parts:
                data = "".join(text_parts)
            else:
                return None
        if not isinstance(data, str):
            return None
        cleaned = data.strip()
        if not cleaned:
            return None
        if cleaned.startswith("```"):
            cleaned_lines: List[str] = []
            for raw_line in cleaned.splitlines():
                line_trimmed = raw_line.strip()
                if line_trimmed.startswith("```"):
                    continue
                cleaned_lines.append(raw_line)
            cleaned = "".join(cleaned_lines).strip()
        try:
            return json.loads(cleaned)
        except json.JSONDecodeError:
            return None

    all_relations: List[Dict] = []
    processed_rows_count = 0
    rows_with_valid_json = 0
    rows_with_edges = 0

    for original_idx, row_data in df_with_results.iterrows():
        processed_rows_count += 1
        json_content = row_data.get(json_result_column)
        raw_publish_date = row_data.get(date_column_name)
        news_title = row_data.get(title_column_name)
        news_body = row_data.get(body_column_name)
        news_id = row_data.get(id_column_name)

        publish_date_str: Optional[str] = None
        if pd.notna(raw_publish_date):
            try:
                publish_date_str = pd.to_datetime(raw_publish_date).strftime("%Y-%m-%d")
            except (ValueError, TypeError):
                publish_date_str = str(raw_publish_date)

        parsed_graph = parse_individual_json_result(json_content)
        if not parsed_graph:
            continue

        rows_with_valid_json += 1
        nodes_list = parsed_graph.get("nodes", [])
        edges_list = parsed_graph.get("edges", [])

        if not isinstance(nodes_list, list) or not isinstance(edges_list, list):
            continue
        if not nodes_list:
            continue

        id_to_label_map: Dict[str, str] = {}
        id_to_summary_map: Dict[str, Optional[str]] = {}

        for node in nodes_list:
            if not isinstance(node, dict) or "id" not in node:
                continue
            node_id_str = str(node["id"])
            id_to_label_map[node_id_str] = node.get("label", node_id_str)
            id_to_summary_map[node_id_str] = node.get("summary")

        current_row_edges = 0
        for edge in edges_list:
            if not isinstance(edge, dict):
                continue
            from_id = edge.get("from")
            to_id = edge.get("to")
            if from_id is None or to_id is None:
                continue
            from_id_str = str(from_id)
            to_id_str = str(to_id)

            relation_entry = {
                "from_node_id": from_id_str,
                "from_node_label": id_to_label_map.get(from_id_str, from_id_str),
                "from_node_summary": id_to_summary_map.get(from_id_str),
                "to_node_id": to_id_str,
                "to_node_label": id_to_label_map.get(to_id_str, to_id_str),
                "to_node_summary": id_to_summary_map.get(to_id_str),
                "relacion_especifica": edge.get("relacion_especifica"),
                "categoria_semantica": edge.get("categoria_semantica"),
                "sentiment": edge.get("sentiment"),
                "news_index": original_idx,
                date_column_name: publish_date_str,
                title_column_name: news_title,
                body_column_name: news_body,
                id_column_name: news_id,
            }
            all_relations.append(relation_entry)
            current_row_edges += 1

        if current_row_edges > 0:
            rows_with_edges += 1

    if not all_relations:
        print("No se pudieron extraer relaciones desde los resultados proporcionados.")
        return pd.DataFrame(
            columns=[
                "from_node_id",
                "from_node_label",
                "from_node_summary",
                "to_node_id",
                "to_node_label",
                "to_node_summary",
                "relacion_especifica",
                "categoria_semantica",
                "sentiment",
                "news_index",
                date_column_name,
                title_column_name,
                body_column_name,
                id_column_name,
            ]
        )

    edgelist_output_df = pd.DataFrame(all_relations)

    print("--- Estadisticas de Extraccion del Edgelist ---")
    print(f"Filas procesadas: {processed_rows_count}")
    print(f"Filas con JSON valido: {rows_with_valid_json}")
    print(f"Filas que generaron al menos un edge: {rows_with_edges}")
    print(f"Total de edges extraidos: {len(edgelist_output_df)}")
    print("--- Fin Estadisticas ---")

    return edgelist_output_df


# --- Logica Principal de la Celda 2 ---
print("Iniciando procesamiento de resultados del batch...")

print(f"Cargando resultados desde: {jsonl_file_path}")
batch_output_json_lines = read_jsonl_results(jsonl_file_path)

if not batch_output_json_lines:
    print("No se cargaron datos del archivo JSONL o esta vacio. Procesamiento detenido.")
else:
    print(f"Se cargaron {len(batch_output_json_lines)} lineas del archivo de resultados.")

    parsed_results_list = []
    for item_index, result_item in enumerate(batch_output_json_lines):
        custom_id = result_item.get("custom_id")
        original_news_index = None

        if custom_id:
            try:
                original_news_index = int(str(custom_id).split("-")[-1])
            except (ValueError, IndexError, TypeError):
                print(f"Advertencia: No se pudo extraer el indice del custom_id '{custom_id}' (item {item_index}).")

        actual_result_content = None
        response_container = result_item.get("response")
        if isinstance(response_container, dict):
            body = response_container.get("body")
            if isinstance(body, dict):
                choices = body.get("choices")
                if isinstance(choices, list) and choices:
                    message = choices[0].get("message")
                    if isinstance(message, dict):
                        actual_result_content = message.get("content")
        if actual_result_content is None and result_item.get("error"):
            error_info = result_item["error"]
            print(f"Advertencia: Error en la respuesta del batch para custom_id '{custom_id}': {error_info}")

        if original_news_index is not None:
            parsed_results_list.append(
                {
                    "original_news_idx": original_news_index,
                    "ia_result_json": actual_result_content,
                }
            )

    if not parsed_results_list:
        print("Critico: No se pudieron procesar resultados validos (con indice). Deteniendo.")
    else:
        batch_results_df = pd.DataFrame(parsed_results_list)
        print(f"Creado batch_results_df con {len(batch_results_df)} filas de resultados de IA.")

        try:
            if "df" not in globals() or not isinstance(df, pd.DataFrame):
                raise NameError("El DataFrame 'df' original no esta disponible. Ejecuta la celda anterior.")

            columnas_requeridas = {
                NOMBRE_COLUMNA_FECHA_NOTICIA: "fecha",
                NOMBRE_COLUMNA_TITULO_NOTICIA: "titulo",
                NOMBRE_COLUMNA_CUERPO_NOTICIA: "cuerpo",
                NOMBRE_COLUMNA_ID_NOTICIA: "id",
            }
            for col_name, descripcion in columnas_requeridas.items():
                if col_name not in df.columns:
                    print(f"Error critico: La columna de {descripcion} '{col_name}' no existe en 'df'.")
                    print(f"Columnas disponibles: {df.columns.tolist()}")
                    raise KeyError(f"Columna '{col_name}' no encontrada en df.")

            print(f"DataFrame 'df' disponible con {len(df)} filas.")

            df_merged_with_results = df.merge(
                batch_results_df,
                left_index=True,
                right_on="original_news_idx",
                how="inner",
            )

            print(f"df_merged_with_results creado con {len(df_merged_with_results)} filas.")

            if "ia_result_json" not in df_merged_with_results.columns:
                print("Error critico: La columna 'ia_result_json' no existe en el DataFrame combinado.")
            elif df_merged_with_results["ia_result_json"].isna().all() and len(parsed_results_list) > 0:
                print("Advertencia: Todas las entradas en 'ia_result_json' son NaN.")

            if df_merged_with_results.empty:
                print("El DataFrame combinado esta vacio. No hay datos para extraer el edgelist.")
            else:
                print("Extrayendo edgelist con el nuevo formato (nodos y relaciones enriquecidas)...")
                final_edgelist_df = extract_network_edgelist_from_results_v2(
                    df_merged_with_results,
                    "ia_result_json",
                    date_column_name=NOMBRE_COLUMNA_FECHA_NOTICIA,
                    title_column_name=NOMBRE_COLUMNA_TITULO_NOTICIA,
                    body_column_name=NOMBRE_COLUMNA_CUERPO_NOTICIA,
                    id_column_name=NOMBRE_COLUMNA_ID_NOTICIA,
                )

                if not final_edgelist_df.empty:
                    column_order = [
                        "from_node_id",
                        "from_node_label",
                        "from_node_summary",
                        "to_node_id",
                        "to_node_label",
                        "to_node_summary",
                        "relacion_especifica",
                        "categoria_semantica",
                        "sentiment",
                        "news_index",
                        NOMBRE_COLUMNA_ID_NOTICIA,
                        NOMBRE_COLUMNA_FECHA_NOTICIA,
                        NOMBRE_COLUMNA_TITULO_NOTICIA,
                        NOMBRE_COLUMNA_CUERPO_NOTICIA,
                    ]
                    existing_columns = [col for col in column_order if col in final_edgelist_df.columns]
                    final_edgelist_df = final_edgelist_df.loc[:, existing_columns]

                    print("Edgelist final extraido con exito. Primeras filas:")
                    print(final_edgelist_df.head())

                    try:
                        final_edgelist_df.to_csv(output_edgelist_csv_path, index=False)
                        print(f"Edgelist guardado en: {output_edgelist_csv_path}")
                    except Exception as exc:
                        print(f"Error al guardar el edgelist en CSV: {exc}")
                else:
                    print("El edgelist resultante esta vacio.")

        except (NameError, KeyError) as error_referencia:
            print(f"Error de referencia o configuracion: {error_referencia}")
            print("No se completo el procesamiento. Verifica que 'df' este en memoria con las columnas requeridas.")
        except Exception as error_general:
            print(f"Ocurrio un error inesperado: {error_general}")
            import traceback
            traceback.print_exc()


Iniciando procesamiento de resultados del batch...
Cargando resultados desde: json/batch_68d4b956f7ec8190a2eff7dc12efcbc0_output.jsonl
Se cargaron 9999 lineas del archivo de resultados.
Creado batch_results_df con 9999 filas de resultados de IA.
DataFrame 'df' disponible con 36654 filas.
df_merged_with_results creado con 9999 filas.
Extrayendo edgelist con el nuevo formato (nodos y relaciones enriquecidas)...
--- Estadisticas de Extraccion del Edgelist ---
Filas procesadas: 9999
Filas con JSON valido: 9999
Filas que generaron al menos un edge: 8971
Total de edges extraidos: 52214
--- Fin Estadisticas ---
Edgelist final extraido con exito. Primeras filas:
  from_node_id from_node_label  \
0            3     Guido Garay   
1            3     Guido Garay   
2            7   Ennio Vivaldi   
3            7   Ennio Vivaldi   
4            7   Ennio Vivaldi   

                                   from_node_summary to_node_id  \
0  Astrónomo que liderará jornada educativa en Ce...          4  

In [ ]:
# Celda 2: Procesamiento de Resultados y Generación de Edgelist (Formato Modificado)

# --- Importaciones para Celda 2 ---
import pandas as pd
import json
from typing import List, Dict, Union, Optional # Para type hints

# --- Configuración ---
# !!! IMPORTANTE: ACTUALIZA ESTA RUTA al archivo de resultados .jsonl descargado de OpenAI !!!
jsonl_file_path = "json/primarias2025.jsonl" # Ejemplo de ruta

df_filtrado = df.copy()

# Nombre del archivo CSV de salida para el edgelist final
output_edgelist_csv_path = "output/primarias2025.csv" # Nombre de archivo modificado

# Nombre de la columna en df_merged_with_results que contiene la fecha de la noticia
NOMBRE_COLUMNA_FECHA_NOTICIA = 'dateTime' # Ajusta si tu columna de fecha se llama diferente

# Nombres de las columnas para título y cuerpo de la noticia
# Asegúrate de que estas columnas existan en el DataFrame 'df' original (proveniente de df_filtrado)
NOMBRE_COLUMNA_TITULO_NOTICIA = 'title' # Asume que la columna se llama 'title'
NOMBRE_COLUMNA_CUERPO_NOTICIA = 'body'  # Asume que la columna se llama 'body'


# --- Funciones Auxiliares ---
def read_jsonl_results(file_path: str) -> List[Dict]:
    """Lee un archivo JSONL de resultados de batch y devuelve una lista de diccionarios."""
    json_objects = []
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line_number, line in enumerate(file, 1):
                line_stripped = line.strip()
                if line_stripped: # Asegurarse de que la línea no esté vacía
                    try:
                        json_objects.append(json.loads(line_stripped))
                    except json.JSONDecodeError as e:
                        print(f"Advertencia: Error decodificando JSON en línea {line_number} del archivo '{file_path}': {e}. Línea: '{line_stripped[:100]}...'")
    except FileNotFoundError:
        print(f"Error CRÍTICO: El archivo de resultados '{file_path}' no se encontró. Por favor, verifica la ruta.")
        raise
    except Exception as e:
        print(f"Error CRÍTICO al leer el archivo '{file_path}': {e}")
        raise
    return json_objects

def extract_network_edgelist_from_results_v2(
    df_with_results: pd.DataFrame,
    json_result_column: str,
    date_column_name: str,
    title_column_name: str, # <--- NUEVO PARÁMETRO
    body_column_name: str,  # <--- NUEVO PARÁMETRO
    node_type_attribute: str = 'type',
    default_node_type: str = 'UnknownNode'
) -> pd.DataFrame:
    """
    Extrae un edgelist con formato extendido desde una columna del DataFrame
    que contiene los resultados JSON de la IA, incluyendo título y cuerpo de la noticia.

    Output edgelist columns:
    FROM_NODE, TO_NODE, TYPE_FROM_NODE, TYPE_TO_NODE, SIGN, TYPE_SIGN, INDEX, DATE, TITLE, BODY
    """
    def parse_individual_json_result(data: Union[str, Dict, None]) -> Optional[Dict]:
        if data is None: return None
        if isinstance(data, dict): return data
        if not isinstance(data, str): return None
        try:
            return json.loads(data)
        except json.JSONDecodeError:
            return None

    all_relations = []
    processed_rows_count = 0
    rows_with_valid_parseable_json = 0
    rows_with_edges = 0

    for original_idx, row_data in df_with_results.iterrows():
        processed_rows_count += 1
        json_content_str = row_data.get(json_result_column)
        news_date = row_data.get(date_column_name)
        news_title = row_data.get(title_column_name) # <--- EXTRAER TÍTULO
        news_body = row_data.get(body_column_name)   # <--- EXTRAER CUERPO

        if pd.notna(news_date) and hasattr(news_date, 'strftime'):
            try:
                news_date_str = news_date.strftime('%Y-%m-%d')
            except AttributeError:
                 news_date_str = str(news_date) if pd.notna(news_date) else None
        else:
            news_date_str = str(news_date) if pd.notna(news_date) else None

        parsed_graph_data = parse_individual_json_result(json_content_str)

        if parsed_graph_data:
            rows_with_valid_parseable_json += 1
            nodes_list = parsed_graph_data.get('nodes', [])
            edges_list = parsed_graph_data.get('edges', [])

            if isinstance(nodes_list, list) and isinstance(edges_list, list) and nodes_list:
                id_to_label_map = {}
                id_to_type_map = {}

                for node in nodes_list:
                    if isinstance(node, dict) and 'id' in node and 'label' in node:
                        node_id_str = str(node['id'])
                        id_to_label_map[node_id_str] = node['label']
                        id_to_type_map[node_id_str] = node.get(node_type_attribute, default_node_type)

                current_row_edges = 0
                for edge in edges_list:
                    if not isinstance(edge, dict): continue

                    from_id_original = edge.get('from')
                    to_id_original = edge.get('to')

                    if from_id_original is None or to_id_original is None: continue

                    from_id_str = str(from_id_original)
                    to_id_str = str(to_id_original)

                    if from_id_str in id_to_label_map and to_id_str in id_to_label_map:
                        all_relations.append({
                            'FROM_NODE': id_to_label_map[from_id_str],
                            'TO_NODE': id_to_label_map[to_id_str],
                            'TYPE_FROM_NODE': id_to_type_map.get(from_id_str, default_node_type),
                            'TYPE_TO_NODE': id_to_type_map.get(to_id_str, default_node_type),
                            'SIGN': edge.get('sentiment', 'neutral'),
                            'TYPE_SIGN': edge.get('label', 'desconocido'),
                            'INDEX': original_idx, # Este es el índice de df_merged_with_results
                            'DATE': news_date_str,
                            'TITLE': news_title, # <--- AÑADIR TÍTULO A LA RELACIÓN
                            'BODY': news_body     # <--- AÑADIR CUERPO A LA RELACIÓN
                        })
                        current_row_edges += 1

                if current_row_edges > 0:
                    rows_with_edges += 1

    final_columns = ['FROM_NODE', 'TO_NODE', 'TYPE_FROM_NODE', 'TYPE_TO_NODE',
                     'SIGN', 'TYPE_SIGN', 'INDEX', 'DATE',
                     'TITLE', 'BODY'] # <--- ACTUALIZAR COLUMNAS FINALES

    edgelist_output_df = pd.DataFrame(all_relations, columns=final_columns) if all_relations \
                         else pd.DataFrame(columns=final_columns)

    total_input_rows = len(df_with_results)
    print(f"\n--- Estadísticas de Extracción de Edgelist (v2) ---")
    print(f"Total de filas de entrada (noticias con resultado): {total_input_rows}")

    ratio_parseable = (rows_with_valid_parseable_json / total_input_rows) if total_input_rows > 0 else 0.0
    print(f"Filas con contenido JSON parseable en '{json_result_column}': {rows_with_valid_parseable_json}/{total_input_rows} ({ratio_parseable:.2%})")

    ratio_edges = (rows_with_edges / total_input_rows) if total_input_rows > 0 else 0.0
    print(f"Filas que produjeron al menos una relación (edge): {rows_with_edges}/{total_input_rows} ({ratio_edges:.2%})")

    print(f"Total de relaciones (edges) extraídas: {len(edgelist_output_df)}")
    print(f"--- Fin Estadísticas ---")

    return edgelist_output_df

# --- Lógica Principal de la Celda 2 ---
print(f"Iniciando procesamiento de resultados del batch...")

# 1. Leer los resultados del archivo JSONL descargado de OpenAI
print(f"Cargando resultados desde: {jsonl_file_path}")
batch_output_json_lines = read_jsonl_results(jsonl_file_path)

if not batch_output_json_lines:
    print("No se cargaron datos del archivo JSONL de resultados o el archivo está vacío. Deteniendo procesamiento.")
else:
    print(f"Se cargaron {len(batch_output_json_lines)} líneas (respuestas) del archivo de resultados del batch.")

    # 2. Procesar las líneas JSON para crear un DataFrame de resultados (batch_results_df)
    parsed_results_list = []
    for item_index, result_item in enumerate(batch_output_json_lines):
        custom_id = result_item.get('custom_id')
        original_news_index = None

        if custom_id:
            try:
                original_news_index = int(custom_id.split('-')[-1])
            except (ValueError, IndexError, TypeError):
                print(f"Advertencia: No se pudo extraer el índice numérico del custom_id: '{custom_id}' en el item {item_index}.")

        actual_result_content = None
        if result_item.get('response') and isinstance(result_item['response'], dict) and \
           result_item['response'].get('body') and isinstance(result_item['response']['body'], dict) and \
           result_item['response']['body'].get('choices') and isinstance(result_item['response']['body']['choices'], list) and \
           len(result_item['response']['body']['choices']) > 0 and \
           isinstance(result_item['response']['body']['choices'][0], dict) and \
           result_item['response']['body']['choices'][0].get('message') and isinstance(result_item['response']['body']['choices'][0]['message'], dict):
            actual_result_content = result_item['response']['body']['choices'][0]['message'].get('content')
        elif result_item.get('error'):
             error_info = result_item['error']
             print(f"Advertencia: Error en la respuesta del batch para custom_id '{custom_id}': {error_info.get('message', str(error_info))}")

        if original_news_index is not None:
            parsed_results_list.append({
                'original_news_idx': original_news_index,
                'ia_result_json': actual_result_content,
            })

    if not parsed_results_list:
        print("CRÍTICO: No se pudieron procesar resultados válidos (con índice) desde las líneas JSON del batch. Deteniendo.")
    else:
        batch_results_df = pd.DataFrame(parsed_results_list)
        print(f"Creado batch_results_df con {len(batch_results_df)} filas (resultados de IA procesados).")

        try:
            # Asegúrate de que 'df' (o 'df_filtrado' de las celdas anteriores) esté disponible
            # y que contenga las columnas de fecha, título y cuerpo.
            # El 'df' aquí se refiere al DataFrame que contiene los datos originales de las noticias.
            # En tu flujo, este 'df' es el cargado en la Celda 1 (o df_filtrado si es el que se mantiene).
            if 'df' not in globals() or not isinstance(df, pd.DataFrame):
                 print("Error CRÍTICO: El DataFrame 'df' original (que contiene las fechas, títulos y cuerpos) no está disponible.")
                 print("Asegúrate de que la Celda 1 se haya ejecutado y que 'df' (o 'df_filtrado') esté en memoria.")
                 raise NameError("'df' original no definido o no es un DataFrame")

            # Verificar si las columnas de fecha, título y cuerpo existen en el DataFrame 'df'
            columnas_requeridas = {
                NOMBRE_COLUMNA_FECHA_NOTICIA: "fecha",
                NOMBRE_COLUMNA_TITULO_NOTICIA: "título",
                NOMBRE_COLUMNA_CUERPO_NOTICIA: "cuerpo"
            }
            for col_name, desc in columnas_requeridas.items():
                if col_name not in df.columns:
                    print(f"Error CRÍTICO: La columna de {desc} '{col_name}' no se encuentra en el DataFrame 'df'.")
                    print(f"Columnas disponibles en 'df': {df.columns.tolist()}")
                    raise KeyError(f"Columna de {desc} '{col_name}' no encontrada en df.")

            print(f"DataFrame 'df' original encontrado en memoria con {len(df)} filas y columnas: {df.columns.tolist()}.")

            df_merged_with_results = df.merge(
                batch_results_df,
                left_index=True, # Usa el índice de 'df' (que debe ser el 'df_filtrado' o su equivalente)
                right_on='original_news_idx',
                how='inner' # Asegura que solo se procesen noticias con resultados y viceversa
            )
            # 'original_news_idx' en df_merged_with_results es el índice original de 'df' antes del merge.
            # Esto es útil si se quiere referenciar de nuevo al 'df' original.
            # El 'INDEX' en el edgelist final será el índice de df_merged_with_results, que corresponde al índice de 'df'.

            print(f"df_merged_with_results creado con {len(df_merged_with_results)} filas después del merge.")

            if 'ia_result_json' not in df_merged_with_results.columns:
                 print("Error CRÍTICO: La columna 'ia_result_json' no está en df_merged_with_results.")
            elif df_merged_with_results['ia_result_json'].isna().all() and len(parsed_results_list) > 0 :
                 print("Advertencia: Todas las entradas en la columna 'ia_result_json' de df_merged_with_results son NaN.")

            if df_merged_with_results.empty:
                print("El DataFrame combinado está vacío. No se puede proceder a extraer el edgelist.")
            else:
                print("Extrayendo edgelist de los resultados combinados con el nuevo formato (incluyendo título y cuerpo)...")
                final_edgelist_df = extract_network_edgelist_from_results_v2(
                    df_merged_with_results,
                    'ia_result_json',
                    date_column_name=NOMBRE_COLUMNA_FECHA_NOTICIA,
                    title_column_name=NOMBRE_COLUMNA_TITULO_NOTICIA, # <--- PASAR NOMBRE DE COLUMNA DE TÍTULO
                    body_column_name=NOMBRE_COLUMNA_CUERPO_NOTICIA   # <--- PASAR NOMBRE DE COLUMNA DE CUERPO
                )

                if not final_edgelist_df.empty:
                    print(f"\nEdgelist final (nuevo formato con contenido) extraído con éxito, contiene {len(final_edgelist_df)} relaciones.")
                    print("Primeras 5 filas del edgelist_df final:")
                    print(final_edgelist_df.head())

                    try:
                        final_edgelist_df.to_csv(output_edgelist_csv_path, index=False)
                        print(f"\nEdgelist final (nuevo formato con contenido) guardado exitosamente en: {output_edgelist_csv_path}")
                    except Exception as e:
                        print(f"Error al guardar el edgelist CSV final: {e}")
                else:
                    print("\nEl edgelist final (nuevo formato con contenido) está vacío.")

        except (NameError, KeyError) as e_ref:
            print(f"Error de referencia o configuración: {e_ref}")
            print("No se pudo completar el procesamiento. Verifica que 'df' esté disponible y contenga las columnas requeridas (fecha, título, cuerpo).")
        except Exception as e_general:
            print(f"Ocurrió un error inesperado: {e_general}")
            import traceback
            traceback.print_exc()

Iniciando procesamiento de resultados del batch...
Cargando resultados desde: json/plebiscito2022.jsonl
Se cargaron 24554 líneas (respuestas) del archivo de resultados del batch.
Creado batch_results_df con 24554 filas (resultados de IA procesados).
DataFrame 'df' original encontrado en memoria con 1314578 filas y columnas: ['title', 'body', 'date', 'url', 'source_title', 'source_dataType', 'source_uri', 'time', 'dateTime', 'mes_año'].
df_merged_with_results creado con 24554 filas después del merge.
Extrayendo edgelist de los resultados combinados con el nuevo formato (incluyendo título y cuerpo)...

--- Estadísticas de Extracción de Edgelist (v2) ---
Total de filas de entrada (noticias con resultado): 24554
Filas con contenido JSON parseable en 'ia_result_json': 24553/24554 (100.00%)
Filas que produjeron al menos una relación (edge): 18146/24554 (73.90%)
Total de relaciones (edges) extraídas: 68056
--- Fin Estadísticas ---

Edgelist final (nuevo formato con contenido) extraído con éxi

In [13]:
df = pd.read_csv('output\edgelist_output_gpt4omini_5k_nuevo_formato_con_contenido.csv')

<>:1: SyntaxWarning: invalid escape sequence '\e'
<>:1: SyntaxWarning: invalid escape sequence '\e'
C:\Users\benja\AppData\Local\Temp\ipykernel_16256\3468456252.py:1: SyntaxWarning: invalid escape sequence '\e'
  df = pd.read_csv('output\edgelist_output_gpt4omini_5k_nuevo_formato_con_contenido.csv')


In [15]:
df

,FROM_NODE,TO_NODE,TYPE_FROM_NODE,TYPE_TO_NODE,SIGN,TYPE_SIGN,INDEX,DATE,TITLE,BODY
0,Verónica Contreras,Verónica Rincón,UnknownNode,UnknownNode,neutral,presented proposal to,0,2022-10-25 23:08:00+00:00,"Juan Pablo Swett, de la Multigremial de Empren...","Junto a Verónica Contreras, presidenta de la C..."
1,Verónica Contreras,Matías Walker,UnknownNode,UnknownNode,neutral,presented proposal to,0,2022-10-25 23:08:00+00:00,"Juan Pablo Swett, de la Multigremial de Empren...","Junto a Verónica Contreras, presidenta de la C..."
2,Swett,Verónica Rincón,UnknownNode,UnknownNode,neutral,shared constitutional expectations,0,2022-10-25 23:08:00+00:00,"Juan Pablo Swett, de la Multigremial de Empren...","Junto a Verónica Contreras, presidenta de la C..."
3,Swett,Matías Walker,UnknownNode,UnknownNode,neutral,shared constitutional expectations,0,2022-10-25 23:08:00+00:00,"Juan Pablo Swett, de la Multigremial de Empren...","Junto a Verónica Contreras, presidenta de la C..."
4,Verónica Contreras,Swett,UnknownNode,UnknownNode,positive,aligned on constitutional process,0,2022-10-25 23:08:00+00:00,"Juan Pablo Swett, de la Multigremial de Empren...","Junto a Verónica Contreras, presidenta de la C..."
...,...,...,...,...,...,...,...,...,...,...
68051,Francisco Muñoz,Ximena Ossandón,UnknownNode,UnknownNode,negative,intimidates parliamentary members,24552,2022-10-05 17:35:00+00:00,Cámara de Diputados aprueba condenar actos y e...,La Cámara de Diputados aprobó este miércoles e...
68052,Francisco Muñoz,Leonidas Romero,UnknownNode,UnknownNode,negative,intimidates parliamentary members,24552,2022-10-05 17:35:00+00:00,Cámara de Diputados aprueba condenar actos y e...,La Cámara de Diputados aprobó este miércoles e...
68053,Gonzalo de la Carrera,Francisco Muñoz,UnknownNode,UnknownNode,negative,votes against condemning violence,24552,2022-10-05 17:35:00+00:00,Cámara de Diputados aprueba condenar actos y e...,La Cámara de Diputados aprobó este miércoles e...
68054,Javier Macaya,Gabriel Boric,UnknownNode,UnknownNode,negative,criticizes for comments on Carabineros,24553,2022-10-19 13:02:00+00:00,Macaya emplaza al Presidente Boric a pedir dis...,"El presidente y senador de la UDI, Javier Maca..."


In [16]:
import pandas as pd


# Concatena las dos columnas en una sola Serie
all_nodes = pd.concat([df['FROM_NODE'], df['TO_NODE']])

# Cuenta la frecuencia de cada nodo y obtén los 20 principales
top_20_nodes = all_nodes.value_counts().head(20)

# Imprime el resultado
print(top_20_nodes)

Gabriel Boric            15114
Ximena Rincón             2068
Giorgio Jackson           2050
Camila Vallejo            1918
Raúl Soto                 1597
Matías Walker             1570
Álvaro Elizalde           1350
Izkia Siches              1326
Karol Cariola             1260
Javier Macaya             1219
Ricardo Lagos             1217
Sebastián Piñera          1190
Michelle Bachelet         1066
Mario Marcel              1051
Carolina Tohá             1028
Francisco Chahuán          961
Guillermo Teillier         874
Paulina Vodanovic          803
José Antonio Kast          770
María Elisa Quinteros      717
Name: count, dtype: int64
